In [ ]:
import numpy as np
from scipy import stats

# ------------------------------------------------------------
# آزمون t تک نمونه‌ای (One-sample t-test)
# ------------------------------------------------------------
class OneSampleTTest:
    def __init__(self, data, null_mean=0, alternative='two-sided'):
        """
        data: آرایه‌ای از داده‌ها
        null_mean: مقدار فرضی میانگین (μ0)
        alternative: نوع آزمون ('two-sided', 'greater', 'less')
        """
        self.data = np.asarray(data)
        self.n = len(self.data)
        self.null_mean = null_mean
        self.alternative = alternative
        self.df = self.n - 1
        self._compute()

    def _compute(self):
        self.mean = np.mean(self.data)
        self.std = np.std(self.data, ddof=1)  # انحراف معیار نمونه
        self.t_stat = (self.mean - self.null_mean) / (self.std / np.sqrt(self.n))
        if self.alternative == 'two-sided':
            self.p_value = 2 * (1 - stats.t.cdf(abs(self.t_stat), self.df))
        elif self.alternative == 'greater':
            self.p_value = 1 - stats.t.cdf(self.t_stat, self.df)
        elif self.alternative == 'less':
            self.p_value = stats.t.cdf(self.t_stat, self.df)
        else:
            raise ValueError("alternative must be 'two-sided', 'greater', or 'less'")

    def summary(self, alpha=0.05):
        print("آزمون t تک نمونه‌ای")
        print(f"داده‌ها: n = {self.n}, میانگین = {self.mean:.4f}, انحراف معیار = {self.std:.4f}")
        print(f"فرض صفر: میانگین = {self.null_mean}")
        print(f"آماره t = {self.t_stat:.4f}, درجه آزادی = {self.df}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر در سطح {alpha}")


# ------------------------------------------------------------
# آزمون z تک نمونه‌ای (One-sample z-test)
# ------------------------------------------------------------
class OneSampleZTest:
    def __init__(self, data, null_mean=0, sigma=None, alternative='two-sided'):
        """
        data: آرایه‌ای از داده‌ها
        null_mean: مقدار فرضی میانگین (μ0)
        sigma: انحراف معیار جامعه (معلوم) - اگر داده نشود، از داده تخمین زده می‌شود (برای نمونه‌های بزرگ)
        alternative: نوع آزمون
        """
        self.data = np.asarray(data)
        self.n = len(self.data)
        self.null_mean = null_mean
        self.alternative = alternative
        if sigma is None:
            # اگر سیگما داده نشود، از انحراف معیار نمونه استفاده می‌کنیم (برای نمونه‌های بزرگ معتبر است)
            self.sigma = np.std(self.data, ddof=1)
        else:
            self.sigma = sigma
        self._compute()

    def _compute(self):
        self.mean = np.mean(self.data)
        self.z_stat = (self.mean - self.null_mean) / (self.sigma / np.sqrt(self.n))
        if self.alternative == 'two-sided':
            self.p_value = 2 * (1 - stats.norm.cdf(abs(self.z_stat)))
        elif self.alternative == 'greater':
            self.p_value = 1 - stats.norm.cdf(self.z_stat)
        elif self.alternative == 'less':
            self.p_value = stats.norm.cdf(self.z_stat)
        else:
            raise ValueError("alternative must be 'two-sided', 'greater', or 'less'")

    def summary(self, alpha=0.05):
        print("آزمون z تک نمونه‌ای")
        print(f"داده‌ها: n = {self.n}, میانگین = {self.mean:.4f}")
        print(f"انحراف معیار جامعه (σ) = {self.sigma:.4f}")
        print(f"فرض صفر: میانگین = {self.null_mean}")
        print(f"آماره z = {self.z_stat:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر در سطح {alpha}")


# ------------------------------------------------------------
# آزمون علامت (Sign test)
# ------------------------------------------------------------
class SignTest:
    def __init__(self, data, null_median=0, alternative='two-sided'):
        """
        داده‌ها: آرایه‌ای از اعداد
        null_median: میانه فرضی
        alternative: نوع آزمون
        """
        self.data = np.asarray(data)
        self.n = len(self.data)
        self.null_median = null_median
        self.alternative = alternative
        self._compute()

    def _compute(self):
        # تفاوت‌ها با میانه فرضی
        diff = self.data - self.null_median
        # حذف مقادیر صفر
        diff = diff[diff != 0]
        self.n_eff = len(diff)
        if self.n_eff == 0:
            raise ValueError("همه تفاوت‌ها صفر هستند. آزمون قابل انجام نیست.")
        # تعداد مثبت‌ها
        n_pos = np.sum(diff > 0)
        # تحت فرض صفر، تعداد مثبت‌ها از توزیع دوجمله‌ای با p=0.5 پیروی می‌کند
        if self.alternative == 'two-sided':
            # p-value = 2 * min(P(X ≤ n_pos), P(X ≥ n_pos))
            p_less = stats.binom.cdf(n_pos, self.n_eff, 0.5)
            p_greater = 1 - stats.binom.cdf(n_pos - 1, self.n_eff, 0.5)
            self.p_value = 2 * min(p_less, p_greater)
        elif self.alternative == 'greater':
            self.p_value = 1 - stats.binom.cdf(n_pos - 1, self.n_eff, 0.5)
        elif self.alternative == 'less':
            self.p_value = stats.binom.cdf(n_pos, self.n_eff, 0.5)
        else:
            raise ValueError("alternative must be 'two-sided', 'greater', or 'less'")
        self.n_pos = n_pos

    def summary(self, alpha=0.05):
        print("آزمون علامت (Sign test) برای میانه")
        print(f"داده‌ها: n = {self.n}, تعداد مؤثر (بدون صفر) = {self.n_eff}")
        print(f"تعداد مقادیر مثبت = {self.n_pos}")
        print(f"فرض صفر: میانه = {self.null_median}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر در سطح {alpha}")


# ------------------------------------------------------------
# آزمون ویلکاکسون تک نمونه‌ای (Wilcoxon signed-rank test)
# ------------------------------------------------------------
class WilcoxonSignedRankTest:
    def __init__(self, data, null_median=0, alternative='two-sided'):
        """
        داده‌ها: آرایه‌ای از اعداد
        null_median: میانه فرضی (یا میانگین در صورت تقارن)
        alternative: نوع آزمون
        """
        self.data = np.asarray(data)
        self.n = len(self.data)
        self.null_median = null_median
        self.alternative = alternative
        self._compute()

    def _compute(self):
        # محاسبه تفاوت‌ها
        diff = self.data - self.null_median
        # حذف صفرها
        diff_nonzero = diff[diff != 0]
        self.n_eff = len(diff_nonzero)
        if self.n_eff == 0:
            raise ValueError("همه تفاوت‌ها صفر هستند. آزمون قابل انجام نیست.")
        # رتبه‌بندی قدرمطلق‌ها
        abs_diff = np.abs(diff_nonzero)
        ranks = stats.rankdata(abs_diff)
        # علامت‌ها
        signs = np.sign(diff_nonzero)
        # آماره W = مجموع رتبه‌های مثبت
        w_plus = np.sum(ranks[signs > 0])
        w_minus = np.sum(ranks[signs < 0])
        self.statistic = min(w_plus, w_minus)  # معمولاً آماره W را کوچک‌ترین مجموع می‌گیرند
        # محاسبه p-value با استفاده از scipy (دقیق یا نرمال)
        # برای نمونه‌های بزرگ می‌توان از تقریب نرمال استفاده کرد، اما scipy به طور خودکار مدیریت می‌کند
        if self.alternative == 'two-sided':
            # دوطرفه: دو برابر احتمال یک‌طرفه
            res = stats.wilcoxon(diff, alternative='two-sided')
            self.p_value = res.pvalue
        elif self.alternative == 'greater':
            res = stats.wilcoxon(diff, alternative='greater')
            self.p_value = res.pvalue
        elif self.alternative == 'less':
            res = stats.wilcoxon(diff, alternative='less')
            self.p_value = res.pvalue
        else:
            raise ValueError("alternative must be 'two-sided', 'greater', or 'less'")

    def summary(self, alpha=0.05):
        print("آزمون ویلکاکسون تک نمونه‌ای (Wilcoxon signed-rank)")
        print(f"داده‌ها: n = {self.n}, تعداد مؤثر (بدون صفر) = {self.n_eff}")
        print(f"فرض صفر: میانه = {self.null_median}")
        print(f"آماره W = {self.statistic:.2f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر در سطح {alpha}")

In [ ]:
import numpy as np
from scipy import stats

# ------------------------------------------------------------
# آزمون t دو نمونه مستقل (دو حالت: pooled و Welch)
# ------------------------------------------------------------
class TwoSampleTTest:
    def __init__(self, data1, data2, null_diff=0, alternative='two-sided', equal_var=True):
        """
        data1, data2: آرایه‌های داده برای دو گروه
        null_diff: تفاوت میانگین‌ها تحت فرض صفر (معمولاً 0)
        alternative: 'two-sided', 'greater', 'less'
        equal_var: اگر True، آزمون با فرض برابری واریانس‌ها (pooled)، در غیر این صورت Welch
        """
        self.data1 = np.asarray(data1)
        self.data2 = np.asarray(data2)
        self.n1 = len(self.data1)
        self.n2 = len(self.data2)
        self.null_diff = null_diff
        self.alternative = alternative
        self.equal_var = equal_var

        self.mean1 = np.mean(self.data1)
        self.mean2 = np.mean(self.data2)
        self.var1 = np.var(self.data1, ddof=1)
        self.var2 = np.var(self.data2, ddof=1)

        self._compute()

    def _compute(self):
        # استفاده از ttest_ind از scipy با تنظیمات مناسب
        if self.equal_var:
            res = stats.ttest_ind(self.data1, self.data2, equal_var=True, alternative=self.alternative)
            self.df = self.n1 + self.n2 - 2
            # محاسبه واریانس ترکیبی (pooled variance) برای نمایش
            self.pooled_var = ((self.n1 - 1) * self.var1 + (self.n2 - 1) * self.var2) / self.df
            self.statistic = res.statistic
            self.p_value = res.pvalue
        else:
            res = stats.ttest_ind(self.data1, self.data2, equal_var=False, alternative=self.alternative)
            # درجه آزادی Welch از فرمول تقریبی
            s1 = self.var1 / self.n1
            s2 = self.var2 / self.n2
            self.df = (s1 + s2) ** 2 / (s1**2 / (self.n1 - 1) + s2**2 / (self.n2 - 1))
            self.statistic = res.statistic
            self.p_value = res.pvalue

    def summary(self, alpha=0.05):
        test_type = "t-test دو نمونه مستقل (pooled)" if self.equal_var else "t-test دو نمونه مستقل (Welch)"
        print(test_type)
        print(f"گروه 1: n = {self.n1}, میانگین = {self.mean1:.4f}, واریانس = {self.var1:.4f}")
        print(f"گروه 2: n = {self.n2}, میانگین = {self.mean2:.4f}, واریانس = {self.var2:.4f}")
        print(f"تفاوت میانگین‌ها: {self.mean1 - self.mean2:.4f}")
        print(f"فرض صفر: تفاوت میانگین‌ها = {self.null_diff}")
        print(f"آماره t = {self.statistic:.4f}, درجه آزادی = {self.df:.2f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر در سطح {alpha}")


# ------------------------------------------------------------
# آزمون من-ویتنی U (Mann-Whitney U test)
# ------------------------------------------------------------
class MannWhitneyUTest:
    def __init__(self, data1, data2, alternative='two-sided', use_continuity=True, method='auto'):
        """
        data1, data2: آرایه‌های داده
        alternative: 'two-sided', 'greater', 'less'
        use_continuity: استفاده از تصحیح پیوستگی برای تقریب نرمال
        method: 'auto', 'asymptotic', 'exact'
        """
        self.data1 = np.asarray(data1)
        self.data2 = np.asarray(data2)
        self.alternative = alternative
        self.use_continuity = use_continuity
        self.method = method

        self.n1 = len(self.data1)
        self.n2 = len(self.data2)
        self._compute()

    def _compute(self):
        res = stats.mannwhitneyu(self.data1, self.data2, alternative=self.alternative,
                                 use_continuity=self.use_continuity, method=self.method)
        self.statistic = res.statistic
        self.p_value = res.pvalue

    def summary(self, alpha=0.05):
        print("آزمون من-ویتنی U (Mann-Whitney U)")
        print(f"گروه 1: n = {self.n1}")
        print(f"گروه 2: n = {self.n2}")
        print(f"آماره U = {self.statistic:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر (توزیع‌ها متفاوت هستند) در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر (توزیع‌ها یکسان هستند) در سطح {alpha}")


# ------------------------------------------------------------
# آزمون کولموگروف-اسمیرنوف دو نمونه‌ای (Two-sample Kolmogorov-Smirnov test)
# ------------------------------------------------------------
class TwoSampleKSTest:
    def __init__(self, data1, data2, alternative='two-sided', mode='auto'):
        """
        data1, data2: آرایه‌های داده
        alternative: 'two-sided', 'less', 'greater'
        mode: 'auto', 'exact', 'asymp'
        """
        self.data1 = np.asarray(data1)
        self.data2 = np.asarray(data2)
        self.alternative = alternative
        self.mode = mode

        self.n1 = len(self.data1)
        self.n2 = len(self.data2)
        self._compute()

    def _compute(self):
        res = stats.ks_2samp(self.data1, self.data2, alternative=self.alternative, mode=self.mode)
        self.statistic = res.statistic
        self.p_value = res.pvalue

    def summary(self, alpha=0.05):
        print("آزمون کولموگروف-اسمیرنوف دو نمونه‌ای (Kolmogorov-Smirnov)")
        print(f"گروه 1: n = {self.n1}")
        print(f"گروه 2: n = {self.n2}")
        print(f"آماره KS = {self.statistic:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر (توزیع‌ها متفاوت هستند) در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر (توزیع‌ها یکسان هستند) در سطح {alpha}")


# ------------------------------------------------------------
# مثال استفاده
# ------------------------------------------------------------
if __name__ == "__main__":
    # تولید داده‌های نمونه
    np.random.seed(123)
    group1 = np.random.normal(loc=5, scale=2, size=30)
    group2 = np.random.normal(loc=6, scale=2, size=35)

    # آزمون t pooled
    print("=" * 50)
    ttest_pooled = TwoSampleTTest(group1, group2, null_diff=0, alternative='two-sided', equal_var=True)
    ttest_pooled.summary(alpha=0.05)

    # آزمون t Welch
    print("\n" + "=" * 50)
    ttest_welch = TwoSampleTTest(group1, group2, null_diff=0, alternative='two-sided', equal_var=False)
    ttest_welch.summary(alpha=0.05)

    # آزمون من-ویتنی U
    print("\n" + "=" * 50)
    mw_test = MannWhitneyUTest(group1, group2, alternative='two-sided')
    mw_test.summary(alpha=0.05)

    # آزمون کولموگروف-اسمیرنوف
    print("\n" + "=" * 50)
    ks_test = TwoSampleKSTest(group1, group2, alternative='two-sided')
    ks_test.summary(alpha=0.05)

In [ ]:
import numpy as np
from scipy import stats

# تلاش برای import statsmodels (برای ANOVA با اندازه‌های تکراری)
try:
    from statsmodels.stats.anova import AnovaRM
    STATSMODELS_AVAILABLE = True
except ImportError:
    STATSMODELS_AVAILABLE = False

# ------------------------------------------------------------
# آزمون t زوجی (Paired t-test)
# ------------------------------------------------------------
class PairedTTest:
    def __init__(self, data1, data2, null_diff=0, alternative='two-sided'):
        """
        data1, data2: آرایه‌های داده (پیش و پس یا زوجی)
        null_diff: تفاوت میانگین‌ها تحت فرض صفر (معمولاً 0)
        alternative: 'two-sided', 'greater', 'less'
        """
        self.data1 = np.asarray(data1)
        self.data2 = np.asarray(data2)
        if len(self.data1) != len(self.data2):
            raise ValueError("طول دو گروه باید برابر باشد")
        self.n = len(self.data1)
        self.null_diff = null_diff
        self.alternative = alternative

        self.diff = self.data1 - self.data2
        self.mean_diff = np.mean(self.diff)
        self.std_diff = np.std(self.diff, ddof=1)

        self._compute()

    def _compute(self):
        res = stats.ttest_1samp(self.diff, popmean=self.null_diff, alternative=self.alternative)
        self.statistic = res.statistic
        self.p_value = res.pvalue
        self.df = self.n - 1

    def summary(self, alpha=0.05):
        print("آزمون t زوجی (Paired t-test)")
        print(f"تعداد زوج‌ها: {self.n}")
        print(f"میانگین تفاوت‌ها: {self.mean_diff:.4f}")
        print(f"انحراف معیار تفاوت‌ها: {self.std_diff:.4f}")
        print(f"فرض صفر: تفاوت میانگین‌ها = {self.null_diff}")
        print(f"آماره t = {self.statistic:.4f}, درجه آزادی = {self.df}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر در سطح {alpha}")


# ------------------------------------------------------------
# آزمون ویلکاکسون زوجی (Wilcoxon signed-rank test for paired samples)
# ------------------------------------------------------------
class PairedWilcoxon:
    def __init__(self, data1, data2, null_diff=0, alternative='two-sided'):
        """
        data1, data2: آرایه‌های داده (زوجی)
        null_diff: مقدار فرضی برای تفاوت (معمولاً 0)
        alternative: 'two-sided', 'greater', 'less'
        """
        self.data1 = np.asarray(data1)
        self.data2 = np.asarray(data2)
        if len(self.data1) != len(self.data2):
            raise ValueError("طول دو گروه باید برابر باشد")
        self.n = len(self.data1)
        self.null_diff = null_diff
        self.alternative = alternative

        self.diff = self.data1 - self.data2 - null_diff
        self._compute()

    def _compute(self):
        # حذف تفاوت‌های صفر
        diff_nonzero = self.diff[self.diff != 0]
        self.n_eff = len(diff_nonzero)
        if self.n_eff == 0:
            raise ValueError("همه تفاوت‌ها صفر هستند. آزمون قابل انجام نیست.")

        res = stats.wilcoxon(self.data1, self.data2, zero_method='wilcox', correction=False,
                             alternative=self.alternative)
        self.statistic = res.statistic
        self.p_value = res.pvalue

    def summary(self, alpha=0.05):
        print("آزمون ویلکاکسون زوجی (Wilcoxon signed-rank)")
        print(f"تعداد زوج‌ها: {self.n}, تعداد مؤثر (بدون صفر) = {self.n_eff}")
        print(f"آماره W = {self.statistic:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر در سطح {alpha}")


# ------------------------------------------------------------
# آزمون ANOVA یک‌طرفه (One-way ANOVA)
# ------------------------------------------------------------
class OneWayANOVA:
    def __init__(self, *groups):
        """
        groups: حداقل دو آرایه از داده‌ها برای گروه‌های مختلف
        """
        self.groups = [np.asarray(g) for g in groups]
        self.k = len(self.groups)
        if self.k < 2:
            raise ValueError("حداقل دو گروه لازم است")
        self._compute()

    def _compute(self):
        res = stats.f_oneway(*self.groups)
        self.statistic = res.statistic
        self.p_value = res.pvalue
        self.df_between = self.k - 1
        self.df_within = sum(len(g) for g in self.groups) - self.k

    def summary(self, alpha=0.05):
        print("تحلیل واریانس یک‌طرفه (One-way ANOVA)")
        print(f"تعداد گروه‌ها: {self.k}")
        for i, g in enumerate(self.groups):
            print(f"  گروه {i+1}: n = {len(g)}, میانگین = {np.mean(g):.4f}")
        print(f"آماره F = {self.statistic:.4f}")
        print(f"درجه آزادی بین گروهی: {self.df_between}, درون گروهی: {self.df_within}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: تفاوت معنی‌دار بین گروه‌ها در سطح {alpha}")
        else:
            print(f"نتیجه: عدم تفاوت معنی‌دار بین گروه‌ها در سطح {alpha}")


# ------------------------------------------------------------
# آزمون ANOVA با اندازه‌های تکراری (Repeated measures ANOVA)
# نیازمند statsmodels
# ------------------------------------------------------------
class RepeatedMeasuresANOVA:
    def __init__(self, data, subject_col, measure_col, condition_col):
        """
        data: DataFrame از pandas شامل داده‌ها
        subject_col: نام ستون شناسایی آزمودنی‌ها
        measure_col: نام ستون مقادیر اندازه‌گیری شده
        condition_col: نام ستون شرایط (تکرارها)
        """
        if not STATSMODELS_AVAILABLE:
            raise ImportError("statsmodels نصب نیست. برای اجرای این آزمون، ابتدا آن را نصب کنید: pip install statsmodels")
        import pandas as pd
        if not isinstance(data, pd.DataFrame):
            raise ValueError("داده باید به صورت DataFrame از pandas باشد")
        self.data = data
        self.subject_col = subject_col
        self.measure_col = measure_col
        self.condition_col = condition_col
        self._compute()

    def _compute(self):
        anova_rm = AnovaRM(self.data, self.measure_col, self.subject_col, within=[self.condition_col])
        self.results = anova_rm.fit()
        # استخراج آماره و p-value
        self.statistic = self.results.anova_table['F Value'][0]
        self.p_value = self.results.anova_table['Pr > F'][0]
        self.df_num = self.results.anova_table['Num DF'][0]
        self.df_den = self.results.anova_table['Den DF'][0]

    def summary(self, alpha=0.05):
        print("تحلیل واریانس با اندازه‌های تکراری (Repeated measures ANOVA)")
        print(f"ستون آزمودنی: {self.subject_col}")
        print(f"ستون اندازه‌گیری: {self.measure_col}")
        print(f"ستون شرایط: {self.condition_col}")
        print(f"آماره F = {self.statistic:.4f}")
        print(f"درجه آزادی: ({self.df_num}, {self.df_den})")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: اثر معنی‌دار شرایط در سطح {alpha}")
        else:
            print(f"نتیجه: عدم اثر معنی‌دار شرایط در سطح {alpha}")


# ------------------------------------------------------------
# آزمون کروسکال-والیس (Kruskal-Wallis test)
# ------------------------------------------------------------
class KruskalWallis:
    def __init__(self, *groups):
        """
        groups: حداقل دو آرایه از داده‌ها
        """
        self.groups = [np.asarray(g) for g in groups]
        self.k = len(self.groups)
        if self.k < 2:
            raise ValueError("حداقل دو گروه لازم است")
        self._compute()

    def _compute(self):
        res = stats.kruskal(*self.groups)
        self.statistic = res.statistic
        self.p_value = res.pvalue

    def summary(self, alpha=0.05):
        print("آزمون کروسکال-والیس (Kruskal-Wallis)")
        print(f"تعداد گروه‌ها: {self.k}")
        for i, g in enumerate(self.groups):
            print(f"  گروه {i+1}: n = {len(g)}, میانه = {np.median(g):.4f}")
        print(f"آماره H = {self.statistic:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: تفاوت معنی‌دار بین گروه‌ها در سطح {alpha}")
        else:
            print(f"نتیجه: عدم تفاوت معنی‌دار بین گروه‌ها در سطح {alpha}")


# ------------------------------------------------------------
# آزمون فریدمن (Friedman test)
# ------------------------------------------------------------
class FriedmanTest:
    def __init__(self, *groups):
        """
        groups: آرایه‌های داده برای تکرارها (هر آرایه یک تکرار). همه باید طول یکسان داشته باشند.
        """
        self.groups = [np.asarray(g) for g in groups]
        self.k = len(self.groups)
        if self.k < 2:
            raise ValueError("حداقل دو تکرار لازم است")
        lengths = [len(g) for g in self.groups]
        if not all(l == lengths[0] for l in lengths):
            raise ValueError("همه گروه‌ها باید طول یکسان داشته باشند")
        self.n = lengths[0]
        self._compute()

    def _compute(self):
        res = stats.friedmanchisquare(*self.groups)
        self.statistic = res.statistic
        self.p_value = res.pvalue

    def summary(self, alpha=0.05):
        print("آزمون فریدمن (Friedman test)")
        print(f"تعداد تکرارها (شرایط): {self.k}")
        print(f"تعداد بلوک‌ها (آزمودنی‌ها): {self.n}")
        for i, g in enumerate(self.groups):
            print(f"  تکرار {i+1}: میانگین رتبه = {np.mean(g):.4f} (داده اصلی)")
        print(f"آماره Q = {self.statistic:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: تفاوت معنی‌دار بین تکرارها در سطح {alpha}")
        else:
            print(f"نتیجه: عدم تفاوت معنی‌دار بین تکرارها در سطح {alpha}")

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import chi2, f, norm

# ------------------------------------------------------------
# آزمون z برای یک نسبت (One-proportion z-test)
# ------------------------------------------------------------
class OneProportionZTest:
    def __init__(self, successes, n, p_null, alternative='two-sided', correct=False):
        """
        successes: تعداد موفقیت‌ها (یا آرایه‌ای از داده‌های 0/1)
        n: تعداد کل مشاهدات (اگر successes عدد باشد)
        p_null: نسبت فرضی تحت فرض صفر
        alternative: 'two-sided', 'greater', 'less'
        correct: اعمال تصحیح پیوستگی یتس (Yates' continuity correction)
        """
        if isinstance(successes, (list, np.ndarray)):
            # داده‌های خام (0/1) داده شده
            data = np.asarray(successes)
            self.successes = np.sum(data)
            self.n = len(data)
        else:
            self.successes = successes
            self.n = n
        self.p_null = p_null
        self.alternative = alternative
        self.correct = correct
        self.p_hat = self.successes / self.n
        self._compute()

    def _compute(self):
        se = np.sqrt(self.p_null * (1 - self.p_null) / self.n)
        if self.correct:
            # تصحیح پیوستگی: 0.5/n به تفاوت اضافه می‌شود
            diff = abs(self.p_hat - self.p_null) - 0.5 / self.n
            if diff < 0:
                diff = 0
            self.z_stat = diff / se
            # علامت را با توجه به جهت حفظ می‌کنیم
            if self.p_hat > self.p_null:
                self.z_stat = diff / se
            else:
                self.z_stat = -diff / se
        else:
            self.z_stat = (self.p_hat - self.p_null) / se

        if self.alternative == 'two-sided':
            self.p_value = 2 * (1 - norm.cdf(abs(self.z_stat)))
        elif self.alternative == 'greater':
            self.p_value = 1 - norm.cdf(self.z_stat)
        elif self.alternative == 'less':
            self.p_value = norm.cdf(self.z_stat)
        else:
            raise ValueError("alternative must be 'two-sided', 'greater', or 'less'")

    def summary(self, alpha=0.05):
        print("آزمون z برای یک نسبت (One-proportion z-test)")
        print(f"تعداد موفقیت‌ها: {self.successes} از {self.n}")
        print(f"نسبت مشاهده‌شده: {self.p_hat:.4f}")
        print(f"نسبت فرضی (p0): {self.p_null}")
        print(f"آماره z = {self.z_stat:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر در سطح {alpha}")


# ------------------------------------------------------------
# آزمون z برای دو نسبت (Two-proportion z-test)
# ------------------------------------------------------------
class TwoProportionZTest:
    def __init__(self, successes1, n1, successes2, n2, alternative='two-sided', correct=False):
        """
        successes1, n1: تعداد موفقیت‌ها و تعداد کل گروه اول
        successes2, n2: گروه دوم
        alternative: 'two-sided', 'greater', 'less'
        correct: اعمال تصحیح پیوستگی
        """
        self.successes1 = successes1
        self.n1 = n1
        self.successes2 = successes2
        self.n2 = n2
        self.alternative = alternative
        self.correct = correct
        self.p1_hat = successes1 / n1
        self.p2_hat = successes2 / n2
        self.pooled = (successes1 + successes2) / (n1 + n2)
        self._compute()

    def _compute(self):
        se = np.sqrt(self.pooled * (1 - self.pooled) * (1/self.n1 + 1/self.n2))
        diff = self.p1_hat - self.p2_hat
        if self.correct:
            # تصحیح پیوستگی
            correction = 0.5 * (1/self.n1 + 1/self.n2)
            if abs(diff) > correction:
                diff_adj = abs(diff) - correction
                if diff > 0:
                    diff_adj = diff_adj
                else:
                    diff_adj = -diff_adj
            else:
                diff_adj = 0
            self.z_stat = diff_adj / se
        else:
            self.z_stat = diff / se

        if self.alternative == 'two-sided':
            self.p_value = 2 * (1 - norm.cdf(abs(self.z_stat)))
        elif self.alternative == 'greater':
            self.p_value = 1 - norm.cdf(self.z_stat)
        elif self.alternative == 'less':
            self.p_value = norm.cdf(self.z_stat)
        else:
            raise ValueError("alternative must be 'two-sided', 'greater', or 'less'")

    def summary(self, alpha=0.05):
        print("آزمون z برای دو نسبت (Two-proportion z-test)")
        print(f"گروه 1: {self.successes1} موفقیت از {self.n1} (نسبت {self.p1_hat:.4f})")
        print(f"گروه 2: {self.successes2} موفقیت از {self.n2} (نسبت {self.p2_hat:.4f})")
        print(f"نسبت تلفیقی: {self.pooled:.4f}")
        print(f"آماره z = {self.z_stat:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر (برابری نسبتها) در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر (برابری نسبتها) در سطح {alpha}")


# ------------------------------------------------------------
# آزمون دقیق فیشر (Fisher's exact test)
# ------------------------------------------------------------
class FisherExactTest:
    def __init__(self, table, alternative='two-sided'):
        """
        table: جدول 2×2 به صورت لیست لیست یا آرایه ([[a,b],[c,d]])
        alternative: 'two-sided', 'greater', 'less'
        """
        self.table = np.asarray(table)
        if self.table.shape != (2, 2):
            raise ValueError("جدول باید 2×2 باشد")
        self.alternative = alternative
        self._compute()

    def _compute(self):
        oddsratio, p_value = stats.fisher_exact(self.table, alternative=self.alternative)
        self.oddsratio = oddsratio
        self.p_value = p_value

    def summary(self, alpha=0.05):
        print("آزمون دقیق فیشر (Fisher's exact test)")
        print("جدول 2×2:")
        print(f"  [{self.table[0,0]}, {self.table[0,1]}]")
        print(f"  [{self.table[1,0]}, {self.table[1,1]}]")
        print(f"نسبت شانس (odds ratio) = {self.oddsratio:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر (عدم استقلال) در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر (استقلال) در سطح {alpha}")


# ------------------------------------------------------------
# آزمون کای‌دو برای همگنی (Chi-square test of homogeneity)
# ------------------------------------------------------------
class ChiSquareHomogeneity:
    def __init__(self, table):
        """
        table: جدول توافقی (مشاهدات) با هر ابعادی (حداقل 2×2)
        """
        self.table = np.asarray(table)
        if self.table.ndim != 2:
            raise ValueError("جدول باید دو بعدی باشد")
        self._compute()

    def _compute(self):
        # chi2_contingency آزمون استقلال را انجام می‌دهد که معادل همگنی است
        self.chi2, self.p_value, self.df, self.expected = stats.chi2_contingency(self.table)

    def summary(self, alpha=0.05):
        print("آزمون کای‌دو برای همگنی (Chi-square test of homogeneity)")
        print(f"آماره کای‌دو = {self.chi2:.4f}")
        print(f"درجه آزادی = {self.df}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر (توزیع‌ها همگن نیستند) در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر (توزیع‌ها همگن هستند) در سطح {alpha}")


# ------------------------------------------------------------
# آزمون کای‌دو برای استقلال (Chi-square test of independence)
# ------------------------------------------------------------
class ChiSquareIndependence:
    def __init__(self, table):
        """
        table: جدول توافقی (مشاهدات) با هر ابعادی
        """
        self.table = np.asarray(table)
        if self.table.ndim != 2:
            raise ValueError("جدول باید دو بعدی باشد")
        self._compute()

    def _compute(self):
        self.chi2, self.p_value, self.df, self.expected = stats.chi2_contingency(self.table)

    def summary(self, alpha=0.05):
        print("آزمون کای‌دو برای استقلال (Chi-square test of independence)")
        print(f"آماره کای‌دو = {self.chi2:.4f}")
        print(f"درجه آزادی = {self.df}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر (وابستگی بین متغیرها) در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر (استقلال) در سطح {alpha}")


# ------------------------------------------------------------
# آزمون کای‌دو برای واریانس یک جامعه (Chi-square test for variance)
# ------------------------------------------------------------
class ChiSquareVarianceTest:
    def __init__(self, data, sigma0_sq, alternative='two-sided'):
        """
        data: آرایه‌ای از داده‌ها
        sigma0_sq: واریانس فرضی (σ0^2)
        alternative: 'two-sided', 'greater', 'less'
        """
        self.data = np.asarray(data)
        self.n = len(self.data)
        self.sigma0_sq = sigma0_sq
        self.alternative = alternative
        self.sample_var = np.var(self.data, ddof=1)
        self._compute()

    def _compute(self):
        self.chi2_stat = (self.n - 1) * self.sample_var / self.sigma0_sq
        if self.alternative == 'two-sided':
            self.p_value = 2 * min(chi2.cdf(self.chi2_stat, self.n-1),
                                    1 - chi2.cdf(self.chi2_stat, self.n-1))
        elif self.alternative == 'greater':
            self.p_value = 1 - chi2.cdf(self.chi2_stat, self.n-1)
        elif self.alternative == 'less':
            self.p_value = chi2.cdf(self.chi2_stat, self.n-1)
        else:
            raise ValueError("alternative must be 'two-sided', 'greater', or 'less'")

    def summary(self, alpha=0.05):
        print("آزمون کای‌دو برای واریانس یک جامعه (Chi-square test for variance)")
        print(f"حجم نمونه: {self.n}")
        print(f"واریانس نمونه: {self.sample_var:.4f}")
        print(f"واریانس فرضی (σ0^2): {self.sigma0_sq}")
        print(f"آماره کای‌دو = {self.chi2_stat:.4f}, درجه آزادی = {self.n-1}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر در سطح {alpha}")


# ------------------------------------------------------------
# آزمون F برای برابری دو واریانس (F-test of equality of variances)
# ------------------------------------------------------------
class FTestVariances:
    def __init__(self, data1, data2, alternative='two-sided'):
        """
        data1, data2: آرایه‌های داده برای دو گروه
        alternative: 'two-sided', 'greater', 'less' (معمولاً دوطرفه)
        """
        self.data1 = np.asarray(data1)
        self.data2 = np.asarray(data2)
        self.n1 = len(self.data1)
        self.n2 = len(self.data2)
        self.var1 = np.var(self.data1, ddof=1)
        self.var2 = np.var(self.data2, ddof=1)
        self.alternative = alternative
        self._compute()

    def _compute(self):
        # آماره F = واریانس بزرگتر / واریانس کوچکتر، اما برای یکطرفه باید جهت را مشخص کرد
        # برای سادگی، نسبت واریانس‌ها را به گونه‌ای محاسبه می‌کنیم که >1 شود
        if self.alternative == 'two-sided':
            self.F_stat = self.var1 / self.var2
            # برای دوطرفه، p-value دو برابر احتمال یک‌طرفه است
            p_upper = 1 - f.cdf(self.F_stat, self.n1-1, self.n2-1)
            p_lower = f.cdf(self.F_stat, self.n1-1, self.n2-1)
            self.p_value = 2 * min(p_upper, p_lower)
        elif self.alternative == 'greater':
            # H1: var1 > var2
            self.F_stat = self.var1 / self.var2
            self.p_value = 1 - f.cdf(self.F_stat, self.n1-1, self.n2-1)
        elif self.alternative == 'less':
            # H1: var1 < var2
            self.F_stat = self.var2 / self.var1
            self.p_value = 1 - f.cdf(self.F_stat, self.n2-1, self.n1-1)
        else:
            raise ValueError("alternative must be 'two-sided', 'greater', or 'less'")

    def summary(self, alpha=0.05):
        print("آزمون F برای برابری دو واریانس (F-test)")
        print(f"گروه 1: n={self.n1}, واریانس={self.var1:.4f}")
        print(f"گروه 2: n={self.n2}, واریانس={self.var2:.4f}")
        print(f"آماره F = {self.F_stat:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر (واریانس‌ها برابر نیستند) در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر (واریانس‌ها برابر هستند) در سطح {alpha}")


# ------------------------------------------------------------
# آزمون لون (Levene's test) برای برابری واریانس‌ها در چند گروه
# ------------------------------------------------------------
class LeveneTest:
    def __init__(self, *groups, center='median'):
        """
        groups: حداقل دو آرایه داده
        center: 'median', 'mean', 'trimmed' - آماره مورد استفاده
        """
        self.groups = [np.asarray(g) for g in groups]
        self.k = len(self.groups)
        self.center = center
        self._compute()

    def _compute(self):
        self.statistic, self.p_value = stats.levene(*self.groups, center=self.center)

    def summary(self, alpha=0.05):
        print("آزمون لون (Levene's test) برای برابری واریانس‌ها")
        print(f"تعداد گروه‌ها: {self.k}")
        print(f"آماره آزمون = {self.statistic:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر (واریانس‌ها برابر نیستند) در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر (واریانس‌ها برابر هستند) در سطح {alpha}")


# ------------------------------------------------------------
# آزمون بارتلت (Bartlett's test) برای برابری واریانس‌ها در چند گروه
# ------------------------------------------------------------
class BartlettTest:
    def __init__(self, *groups):
        """
        groups: حداقل دو آرایه داده
        """
        self.groups = [np.asarray(g) for g in groups]
        self.k = len(self.groups)
        self._compute()

    def _compute(self):
        self.statistic, self.p_value = stats.bartlett(*self.groups)

    def summary(self, alpha=0.05):
        print("آزمون بارتلت (Bartlett's test) برای برابری واریانس‌ها")
        print(f"تعداد گروه‌ها: {self.k}")
        print(f"آماره کای‌دو = {self.statistic:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر (واریانس‌ها برابر نیستند) در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر (واریانس‌ها برابر هستند) در سطح {alpha}")

In [ ]:
import numpy as np
from scipy import stats
from scipy.stats import t, f, norm, chi2

# ------------------------------------------------------------
# آزمون همبستگی پیرسون (Pearson correlation test)
# ------------------------------------------------------------
class PearsonCorrelationTest:
    def __init__(self, x, y, alternative='two-sided'):
        """
        x, y: آرایه‌های داده
        alternative: 'two-sided', 'greater', 'less'
        """
        self.x = np.asarray(x)
        self.y = np.asarray(y)
        if len(self.x) != len(self.y):
            raise ValueError("طول دو بردار باید برابر باشد")
        self.n = len(self.x)
        self.alternative = alternative
        self._compute()

    def _compute(self):
        self.r, self.p_value = stats.pearsonr(self.x, self.y)
        # pearsonr p-value دوطرفه است، برای یکطرفه باید تنظیم کنیم
        if self.alternative != 'two-sided':
            # t = r * sqrt((n-2)/(1-r^2))
            t_stat = self.r * np.sqrt((self.n - 2) / (1 - self.r**2))
            if self.alternative == 'greater':
                self.p_value = 1 - stats.t.cdf(t_stat, self.n - 2)
            elif self.alternative == 'less':
                self.p_value = stats.t.cdf(t_stat, self.n - 2)
            else:
                raise ValueError("alternative must be 'two-sided', 'greater', or 'less'")

    def summary(self, alpha=0.05):
        print("آزمون همبستگی پیرسون (Pearson correlation)")
        print(f"تعداد نمونه: {self.n}")
        print(f"ضریب همبستگی r = {self.r:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: همبستگی معنی‌دار در سطح {alpha}")
        else:
            print(f"نتیجه: همبستگی معنی‌دار نیست در سطح {alpha}")


# ------------------------------------------------------------
# آزمون همبستگی اسپیرمن (Spearman's rank correlation test)
# ------------------------------------------------------------
class SpearmanCorrelationTest:
    def __init__(self, x, y, alternative='two-sided'):
        self.x = np.asarray(x)
        self.y = np.asarray(y)
        if len(self.x) != len(self.y):
            raise ValueError("طول دو بردار باید برابر باشد")
        self.n = len(self.x)
        self.alternative = alternative
        self._compute()

    def _compute(self):
        self.rho, self.p_value = stats.spearmanr(self.x, self.y, alternative=self.alternative)

    def summary(self, alpha=0.05):
        print("آزمون همبستگی اسپیرمن (Spearman's rank correlation)")
        print(f"تعداد نمونه: {self.n}")
        print(f"ضریب همبستگی ρ = {self.rho:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: همبستگی معنی‌دار در سطح {alpha}")
        else:
            print(f"نتیجه: همبستگی معنی‌دار نیست در سطح {alpha}")


# ------------------------------------------------------------
# آزمون همبستگی کندال (Kendall's tau test)
# ------------------------------------------------------------
class KendallTauTest:
    def __init__(self, x, y, alternative='two-sided'):
        self.x = np.asarray(x)
        self.y = np.asarray(y)
        if len(self.x) != len(self.y):
            raise ValueError("طول دو بردار باید برابر باشد")
        self.n = len(self.x)
        self.alternative = alternative
        self._compute()

    def _compute(self):
        self.tau, self.p_value = stats.kendalltau(self.x, self.y, alternative=self.alternative)

    def summary(self, alpha=0.05):
        print("آزمون همبستگی کندال (Kendall's tau)")
        print(f"تعداد نمونه: {self.n}")
        print(f"ضریب همبستگی τ = {self.tau:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: همبستگی معنی‌دار در سطح {alpha}")
        else:
            print(f"نتیجه: همبستگی معنی‌دار نیست در سطح {alpha}")


# ------------------------------------------------------------
# آزمون F کلی برای رگرسیون خطی ساده (Overall F-test)
# ------------------------------------------------------------
class OverallFTest:
    def __init__(self, x, y):
        """
        آزمون F برای معنی‌داری کل مدل رگرسیون خطی ساده y = β0 + β1 x
        فرض صفر: β1 = 0
        """
        self.x = np.asarray(x)
        self.y = np.asarray(y)
        if len(self.x) != len(self.y):
            raise ValueError("طول دو بردار باید برابر باشد")
        self.n = len(self.x)
        self._compute()

    def _compute(self):
        # محاسبات رگرسیون خطی ساده
        x_mean = np.mean(self.x)
        y_mean = np.mean(self.y)
        Sxx = np.sum((self.x - x_mean)**2)
        Sxy = np.sum((self.x - x_mean) * (self.y - y_mean))
        Syy = np.sum((self.y - y_mean)**2)

        self.beta1 = Sxy / Sxx
        self.beta0 = y_mean - self.beta1 * x_mean

        # مجموع مربعات
        y_pred = self.beta0 + self.beta1 * self.x
        SSR = np.sum((y_pred - y_mean)**2)  # رگرسیون
        SSE = np.sum((self.y - y_pred)**2)  # خطا
        SST = Syy

        # میانگین مربعات
        MSR = SSR / 1  # درجه آزادی رگرسیون = 1
        MSE = SSE / (self.n - 2)  # درجه آزادی خطا = n-2

        self.F_stat = MSR / MSE
        self.p_value = 1 - f.cdf(self.F_stat, 1, self.n - 2)

    def summary(self, alpha=0.05):
        print("آزمون F کلی برای رگرسیون خطی ساده (Overall F-test)")
        print(f"تعداد نمونه: {self.n}")
        print(f"آماره F = {self.F_stat:.4f}")
        print(f"درجه آزادی: (1, {self.n-2})")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: مدل معنی‌دار است (رد فرض صفر β1=0) در سطح {alpha}")
        else:
            print(f"نتیجه: مدل معنی‌دار نیست در سطح {alpha}")


# ------------------------------------------------------------
# آزمون t برای ضرایب رگرسیون (t-test for coefficients)
# ------------------------------------------------------------
class TTestCoefficients:
    def __init__(self, x, y, coeff='slope'):
        """
        آزمون t برای ضریب شیب (slope) یا عرض از مبدأ (intercept)
        coeff: 'slope' یا 'intercept'
        فرض صفر: ضریب = 0
        """
        self.x = np.asarray(x)
        self.y = np.asarray(y)
        if len(self.x) != len(self.y):
            raise ValueError("طول دو بردار باید برابر باشد")
        self.n = len(self.x)
        self.coeff = coeff
        self._compute()

    def _compute(self):
        # محاسبات رگرسیون
        x_mean = np.mean(self.x)
        y_mean = np.mean(self.y)
        Sxx = np.sum((self.x - x_mean)**2)
        Sxy = np.sum((self.x - x_mean) * (self.y - y_mean))
        Syy = np.sum((self.y - y_mean)**2)

        self.beta1 = Sxy / Sxx
        self.beta0 = y_mean - self.beta1 * x_mean

        # خطای استاندارد
        y_pred = self.beta0 + self.beta1 * self.x
        SSE = np.sum((self.y - y_pred)**2)
        MSE = SSE / (self.n - 2)
        se_beta1 = np.sqrt(MSE / Sxx)
        se_beta0 = np.sqrt(MSE * (1/self.n + x_mean**2 / Sxx))

        if self.coeff == 'slope':
            self.estimate = self.beta1
            self.se = se_beta1
            self.t_stat = self.beta1 / se_beta1
            self.df = self.n - 2
        elif self.coeff == 'intercept':
            self.estimate = self.beta0
            self.se = se_beta0
            self.t_stat = self.beta0 / se_beta0
            self.df = self.n - 2
        else:
            raise ValueError("coeff must be 'slope' or 'intercept'")

        # آزمون دوطرفه
        self.p_value = 2 * (1 - t.cdf(abs(self.t_stat), self.df))

    def summary(self, alpha=0.05):
        coeff_name = "شیب" if self.coeff == 'slope' else "عرض از مبدأ"
        print(f"آزمون t برای ضریب {coeff_name} در رگرسیون خطی ساده")
        print(f"تعداد نمونه: {self.n}")
        print(f"تخمین ضریب = {self.estimate:.4f}")
        print(f"خطای استاندارد = {self.se:.4f}")
        print(f"آماره t = {self.t_stat:.4f}, درجه آزادی = {self.df}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: ضریب معنی‌دار است در سطح {alpha}")
        else:
            print(f"نتیجه: ضریب معنی‌دار نیست در سطح {alpha}")


# ------------------------------------------------------------
# آزمون نسبت درستنمایی (Likelihood Ratio Test)
# ------------------------------------------------------------
class LikelihoodRatioTest:
    def __init__(self, logL_full, logL_reduced, df_diff):
        """
        logL_full: لگاریتم درستنمایی مدل کامل
        logL_reduced: لگاریتم درستنمایی مدل کاهش‌یافته
        df_diff: تفاوت درجه آزادی بین دو مدل (تعداد محدودیت‌ها)
        """
        self.logL_full = logL_full
        self.logL_reduced = logL_reduced
        self.df_diff = df_diff
        self._compute()

    def _compute(self):
        self.LR_stat = -2 * (self.logL_reduced - self.logL_full)
        self.p_value = 1 - chi2.cdf(self.LR_stat, self.df_diff)

    def summary(self, alpha=0.05):
        print("آزمون نسبت درستنمایی (Likelihood Ratio Test)")
        print(f"لگاریتم درستنمایی مدل کامل: {self.logL_full:.4f}")
        print(f"لگاریتم درستنمایی مدل کاهش‌یافته: {self.logL_reduced:.4f}")
        print(f"آماره LR = {self.LR_stat:.4f}")
        print(f"درجه آزادی = {self.df_diff}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر (مدل کامل برتر است) در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر (مدل کاهش‌یافته کافی است) در سطح {alpha}")


# ------------------------------------------------------------
# آزمون والد (Wald Test) برای یک پارامتر
# ------------------------------------------------------------
class WaldTest:
    def __init__(self, estimate, se, null_value=0):
        """
        estimate: تخمین پارامتر
        se: خطای استاندارد تخمین
        null_value: مقدار فرض صفر (معمولاً 0)
        """
        self.estimate = estimate
        self.se = se
        self.null_value = null_value
        self._compute()

    def _compute(self):
        self.W_stat = ((self.estimate - self.null_value) / self.se) ** 2
        self.p_value = 1 - chi2.cdf(self.W_stat, 1)  # درجه آزادی 1

    def summary(self, alpha=0.05):
        print("آزمون والد (Wald test) برای یک پارامتر")
        print(f"تخمین پارامتر = {self.estimate:.4f}")
        print(f"خطای استاندارد = {self.se:.4f}")
        print(f"مقدار فرض صفر = {self.null_value}")
        print(f"آماره والد = {self.W_stat:.4f}")
        print(f"p-value = {self.p_value:.6f}")
        if self.p_value < alpha:
            print(f"نتیجه: رد فرض صفر در سطح {alpha}")
        else:
            print(f"نتیجه: عدم رد فرض صفر در سطح {alpha}")